In [1]:
import numpy as np
import ipykernel as ipy
import pandas as pd
import yaml
import argparse
import typing
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score,confusion_matrix,classification_report
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from interpret.provider import InlineProvider
from interpret import set_visualize_provider
set_visualize_provider(InlineProvider())
from interpret.data import ClassHistogram
from interpret import show, preserve, show_link, set_show_addr

In [2]:
df = pd.read_csv('C:\\Users\\ashis\\OneDrive\\Desktop\\MLOps\\churn_model\\data\\processed\\churn_train.csv')
df_test = pd.read_csv('C:\\Users\\ashis\\OneDrive\\Desktop\\MLOps\\churn_model\\data\\processed\\churn_test.csv')
df.head()

,churn,number_vmail_messages,total_day_calls,total_eve_minutes,total_eve_charge,total_intl_minutes,number_customer_service_calls
0,no,37,58,221.9,18.86,12.3,1
1,no,0,59,283.3,24.08,5.8,2
2,no,0,98,188.1,15.99,12.0,2
3,no,31,92,238.3,20.26,10.7,4
4,no,0,64,139.8,11.88,11.6,2


In [15]:
train_x = df[[col for col in df.columns if col not in ['churn']]]
train_y = df['churn']
test_x = df_test[[col for col in df_test.columns if col not in ['churn']]]
test_y = df_test['churn']

print(f'train shape: {train_x.shape}')
print(f'test shape: {test_x.shape}')

train shape: (3400, 6)
test shape: (850, 6)


In [4]:
ebm = ExplainableBoostingClassifier()
ebm.fit(train_x, train_y)

ebm_global = ebm.explain_global(name = 'EBM')
#preserve(ebm_global, 'number_customer_service_calls', 'number_customer_service_calls.html')
set_show_addr(("127.0.0.1", 5000))
show(ebm_global)

C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ashis\AppData\Local\Programs\Python\Python38\lib\site-packages\interpret\visual\udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:5000/1618661691248/ -->

In [17]:
ebm_local = ebm.explain_local(train_x[:20], train_y[:20], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:5000/1618657640352/ -->

In [6]:
hist = ClassHistogram().explain_data(train_x, train_y, name="Train Data")
show(hist)

<!-- http://127.0.0.1:5000/1618499104928/ -->

In [7]:
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
param_test = {'learning_rate': [0.001,0.005,0.01,0.02],
              'interactions': [5,10,15],
              'max_interaction_bins': [10,15,20],
              'max_rounds': [500,1000,1500,2000],
              'min_samples_leaf': [2,3,5],
              'max_leaves': [3,5,10]}
n_HP_points_to_test=10
LGBM_clf = ExplainableBoostingClassifier(random_state=314, n_jobs=-1)
LGBM_gs = RandomizedSearchCV(
    estimator=LGBM_clf,
    param_distributions=param_test,
    n_iter=n_HP_points_to_test,
    scoring="roc_auc",
    cv=3,
    refit=True,
    random_state=314,
    verbose=False,
)
LGBM_gs.fit(train_x, train_y)

RandomizedSearchCV(cv=3,
                   estimator=ExplainableBoostingClassifier(n_jobs=-1,
                                                           random_state=314),
                   param_distributions={'interactions': [5, 10, 15],
                                        'learning_rate': [0.001, 0.005, 0.01,
                                                          0.03],
                                        'max_interaction_bins': [10, 15, 20],
                                        'max_leaves': [3, 5, 10],
                                        'max_rounds': [5000, 10000, 15000,
                                                       20000],
                                        'min_samples_leaf': [2, 3, 5]},
                   random_state=314, scoring='roc_auc', verbose=False)

In [16]:
from interpret import perf
roc = perf.ROC(LGBM_gs.best_estimator_.predict_proba, feature_names=train_x.columns)
test_y = test_y.map({'yes':1,'no':0})
roc_explanation = roc.explain_perf(test_x, test_y)
show(roc_explanation)

<!-- http://127.0.0.1:5000/1618499858976/ -->